## ====== Export Data ====== 

In [8]:
import os
import sys
from google.colab import drive
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from PIL import Image
import _thread
import time
from tqdm import tqdm

drive.mount('/content/Gdrive', force_remount=True)
dir = "/content/Gdrive/My Drive/Colab Notebooks/PRE/Multi-Modalities/"
sys.path.append(dir)

Mounted at /content/Gdrive


Data paths and files directories :

In [9]:
# Train
train_data_path = '/content/Gdrive/My Drive/Colab Notebooks/PRE/Data/ImagesTr/'
train_label_path = '/content/Gdrive/My Drive/Colab Notebooks/PRE/Data/LabelsTr/'
train_dest = '/content/Gdrive/My Drive/Colab Notebooks/PRE/Data/Data/train/'


# Test
test_data_path = '/content/Gdrive/My Drive/Colab Notebooks/PRE/Data/ImagesTs/'
test_dest = '/content/Gdrive/My Drive/Colab Notebooks/PRE/Data/Data/test/'

In [10]:
start = 70
stop = 120
step = 5

def is_nii_file(filename):
    return filename.endswith('nii.gz')

def load_nii(imgPath, labelPath, dest) :
    img_file = nib.load(imgPath)
    img_file = img_file.get_fdata().astype(np.float32)
    
    lab_file = nib.load(labelPath)
    lab_file = lab_file.get_fdata().astype(np.float32)

    for img_slice in range(start, stop, step):
        img = img_file[:, :, img_slice, :].T
        norm = 255/(img.max() - img.min() + 1e-6)
        img *= norm

        lab = lab_file[:, :, img_slice].T
        mask = (lab == 1).astype(float)

        data = [img, mask]
        name = dest + '/' + os.path.basename(imgPath).split('.')[0] + '_'+ str(img_slice)
        np.save(name, data)

def load_nii_data(imgPath, labelPath, interval, dest):
    image_filenames = [os.path.join(imgPath, "BRATS_{}.nii.gz".format(str(x).zfill(3))) for x in range(interval[0], interval[1])]
    label_filenames = [os.path.join(labelPath, "BRATS_{}.nii.gz".format(str(x).zfill(3))) for x in range(1, 500)]

    ind = interval[0]
    loop = tqdm(total = len(image_filenames), position=0, leave=False)
    for img, lab in zip(image_filenames, label_filenames):
        load_nii(img, lab, dest)
        loop.set_description("Loading " + os.path.basename(img).split('.')[0].format(ind))
        loop.update(1)
        ind += 1
    loop.close()


In [5]:
print("\n ===> Exporting Training set")

training_interval = [1, 389] #80%
test_interval = [389, 485] #20%

load_nii_data(train_data_path, train_label_path, training_interval, train_dest)
load_nii_data(train_data_path, train_label_path, test_interval, test_dest)
print("\nSuccessfully exported !")

  0%|          | 0/388 [00:00<?, ?it/s]


 ===> Exporting Training set



Successfully exported !


Check the data :


In [6]:
def check(data_path):
  image_filenames = [os.path.join(data_path, x) for x in os.listdir(data_path)]
  print("===> Checking ", data_path, '\n')
  for img in image_filenames:
      try:
          test = np.load(img, allow_pickle=True)
      except:
          print(img + " cannot be opened")
  print("'", data_path, "'", " is ok !")

In [7]:
check(train_dest)
check(test_dest)

===> Checking  /content/Gdrive/My Drive/Colab Notebooks/PRE/Data/Data/train/ 

' /content/Gdrive/My Drive/Colab Notebooks/PRE/Data/Data/train/ '  is ok !
===> Checking  /content/Gdrive/My Drive/Colab Notebooks/PRE/Data/Data/test/ 

' /content/Gdrive/My Drive/Colab Notebooks/PRE/Data/Data/test/ '  is ok !
